In [1]:
## This file implements neural networks and random forest on p0040presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 56.32% for over-sampling data, 63.22% after improvement.
## And it is 93.33% for combination data, and is 90% after regularization.
## For random forest, the accuracy is 54.02% for over-sampling data and 86.67% for combination data.
## For random forest with cross-validation, the mean accuracy is 60.29% for over-sampling, and 97.13% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0040presabsSTCC_qual.csv')
df.shape

(255, 375)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     2
15     2
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     2
24     2
25     0
26     2
27     0
28     1
29     2
      ..
225    2
226    1
227    1
228    2
229    2
230    1
231    1
232    2
233    1
234    1
235    2
236    1
237    2
238    1
239    0
240    1
241    1
242    0
243    1
244    1
245    2
246    2
247    1
248    2
249    1
250    2
251    1
252    1
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

2    96
1    95
0    64
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,ST,CC,pheno
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,2
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 374)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,ST,CC,pheno
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,5,5,2
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,8,8,0
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 373) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 96), (1, 96), (2, 96)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 46), (1, 10), (2, 42)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/100
201/201 [==============================] - 0s 535us/step - loss: 6.9066 - accuracy: 0.4129 - val_loss: 2.4828 - val_accuracy: 0.4598
Epoch 2/100
201/201 [==============================] - 0s 110us/step - loss: 4.1755 - accuracy: 0.4975 - val_loss: 1.6500 - val_accuracy: 0.5057
Epoch 3/100
201/201 [==============================] - 0s 120us/step - loss: 2.0518 - accuracy: 0.5274 - val_loss: 1.4017 - val_accuracy: 0.4713
Epoch 4/100
201/201 [==============================] - 0s 131us/step - loss: 1.0943 - accuracy: 0.5174 - val_loss: 1.7920 - val_accuracy: 0.4943
Epoch 5/100
201/201 [==============================] - 0s 126us/step - loss: 1.0170 - accuracy: 0.5672 - val_loss: 1.2247 - val_accuracy: 0.4943
Epoch 6/100
201/201 [==============================] - 0s 131us/step - loss: 0.8549 - accuracy: 0.5821 - val_loss: 1.0861 - val_accuracy: 0.5172
Epoch 7/100
201/201 [==============================] - 0s 127us/step - loss: 0.8177 -

Epoch 57/100
201/201 [==============================] - 0s 138us/step - loss: 0.5754 - accuracy: 0.8159 - val_loss: 1.9431 - val_accuracy: 0.5517
Epoch 58/100
201/201 [==============================] - 0s 110us/step - loss: 0.9899 - accuracy: 0.7960 - val_loss: 1.2842 - val_accuracy: 0.5977
Epoch 59/100
201/201 [==============================] - 0s 104us/step - loss: 0.5643 - accuracy: 0.8209 - val_loss: 2.9948 - val_accuracy: 0.5977
Epoch 60/100
201/201 [==============================] - 0s 95us/step - loss: 0.9174 - accuracy: 0.7861 - val_loss: 2.5780 - val_accuracy: 0.5862
Epoch 61/100
201/201 [==============================] - 0s 95us/step - loss: 0.7854 - accuracy: 0.8010 - val_loss: 2.0064 - val_accuracy: 0.5287
Epoch 62/100
201/201 [==============================] - 0s 94us/step - loss: 0.7097 - accuracy: 0.7960 - val_loss: 2.4608 - val_accuracy: 0.5517
Epoch 63/100
201/201 [==============================] - 0s 106us/step - loss: 0.9870 - accuracy: 0.8308 - val_loss: 1.4953 - va

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

87/87 [==============================] - 0s 101us/step
over-sampling test accuracy: 56.32%


In [33]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [34]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 201 samples, validate on 87 samples
Epoch 1/1000
201/201 [==============================] - 0s 610us/step - loss: 4.0239 - accuracy: 0.3433 - val_loss: 3.4126 - val_accuracy: 0.4368
Epoch 2/1000
201/201 [==============================] - 0s 110us/step - loss: 1.8585 - accuracy: 0.4279 - val_loss: 1.6785 - val_accuracy: 0.4598
Epoch 3/1000
201/201 [==============================] - 0s 132us/step - loss: 1.4869 - accuracy: 0.4826 - val_loss: 1.2528 - val_accuracy: 0.4598
Epoch 4/1000
201/201 [==============================] - 0s 130us/step - loss: 1.1718 - accuracy: 0.5224 - val_loss: 1.4353 - val_accuracy: 0.4943
Epoch 5/1000
201/201 [==============================] - 0s 133us/step - loss: 1.0469 - accuracy: 0.5522 - val_loss: 1.0437 - val_accuracy: 0.5172
Epoch 6/1000
201/201 [==============================] - 0s 144us/step - loss: 0.9441 - accuracy: 0.6020 - val_loss: 1.0334 - val_accuracy: 0.5057
Epoch 7/1000
201/201 [==============================] - 0s 127us/step - loss: 0

201/201 [==============================] - 0s 142us/step - loss: 0.5214 - accuracy: 0.8507 - val_loss: 1.3131 - val_accuracy: 0.6207
Epoch 57/1000
201/201 [==============================] - 0s 141us/step - loss: 0.4017 - accuracy: 0.8706 - val_loss: 1.3202 - val_accuracy: 0.5977
Epoch 58/1000
201/201 [==============================] - 0s 115us/step - loss: 0.3084 - accuracy: 0.8905 - val_loss: 1.4236 - val_accuracy: 0.5862
Epoch 59/1000
201/201 [==============================] - 0s 106us/step - loss: 0.3188 - accuracy: 0.8955 - val_loss: 1.4451 - val_accuracy: 0.5862
Epoch 60/1000
201/201 [==============================] - 0s 96us/step - loss: 0.3227 - accuracy: 0.8706 - val_loss: 1.3963 - val_accuracy: 0.6092
Epoch 61/1000
201/201 [==============================] - 0s 115us/step - loss: 0.3004 - accuracy: 0.9005 - val_loss: 1.3159 - val_accuracy: 0.6092
Epoch 62/1000
201/201 [==============================] - 0s 101us/step - loss: 0.3029 - accuracy: 0.8905 - val_loss: 1.3851 - val_acc

Epoch 112/1000
201/201 [==============================] - 0s 107us/step - loss: 0.1848 - accuracy: 0.9303 - val_loss: 1.6344 - val_accuracy: 0.6092
Epoch 113/1000
201/201 [==============================] - 0s 104us/step - loss: 0.2612 - accuracy: 0.9353 - val_loss: 1.6133 - val_accuracy: 0.6207
Epoch 114/1000
201/201 [==============================] - 0s 98us/step - loss: 0.2778 - accuracy: 0.9254 - val_loss: 1.8324 - val_accuracy: 0.6322
Epoch 115/1000
201/201 [==============================] - 0s 103us/step - loss: 0.2266 - accuracy: 0.9254 - val_loss: 1.5610 - val_accuracy: 0.6437
Epoch 116/1000
201/201 [==============================] - 0s 99us/step - loss: 0.1782 - accuracy: 0.9353 - val_loss: 2.4594 - val_accuracy: 0.6207
Epoch 117/1000
201/201 [==============================] - 0s 106us/step - loss: 0.5011 - accuracy: 0.9104 - val_loss: 1.5993 - val_accuracy: 0.6322
Epoch 118/1000
201/201 [==============================] - 0s 103us/step - loss: 0.2228 - accuracy: 0.9005 - val_lo

201/201 [==============================] - 0s 102us/step - loss: 0.2887 - accuracy: 0.9353 - val_loss: 2.6462 - val_accuracy: 0.6437
Epoch 168/1000
201/201 [==============================] - 0s 105us/step - loss: 0.6840 - accuracy: 0.9453 - val_loss: 1.8226 - val_accuracy: 0.6207
Epoch 169/1000
201/201 [==============================] - 0s 102us/step - loss: 0.2177 - accuracy: 0.9154 - val_loss: 2.3653 - val_accuracy: 0.6322
Epoch 170/1000
201/201 [==============================] - 0s 101us/step - loss: 0.2062 - accuracy: 0.9303 - val_loss: 2.6016 - val_accuracy: 0.6207
Epoch 171/1000
201/201 [==============================] - 0s 98us/step - loss: 0.2181 - accuracy: 0.9254 - val_loss: 2.2868 - val_accuracy: 0.6092
Epoch 172/1000
201/201 [==============================] - 0s 114us/step - loss: 0.1853 - accuracy: 0.9403 - val_loss: 2.2372 - val_accuracy: 0.6437
Epoch 173/1000
201/201 [==============================] - 0s 98us/step - loss: 0.1296 - accuracy: 0.9602 - val_loss: 2.2531 - va

201/201 [==============================] - 0s 124us/step - loss: 0.0806 - accuracy: 0.9701 - val_loss: 2.5775 - val_accuracy: 0.6437
Epoch 223/1000
201/201 [==============================] - 0s 116us/step - loss: 0.0948 - accuracy: 0.9701 - val_loss: 2.5212 - val_accuracy: 0.6437
Epoch 224/1000
201/201 [==============================] - 0s 112us/step - loss: 0.0813 - accuracy: 0.9801 - val_loss: 2.6176 - val_accuracy: 0.6322
Epoch 225/1000
201/201 [==============================] - 0s 98us/step - loss: 0.0880 - accuracy: 0.9701 - val_loss: 2.6770 - val_accuracy: 0.6322
Epoch 226/1000
201/201 [==============================] - 0s 105us/step - loss: 0.0832 - accuracy: 0.9751 - val_loss: 2.6208 - val_accuracy: 0.6437
Epoch 227/1000
201/201 [==============================] - 0s 97us/step - loss: 0.0878 - accuracy: 0.9751 - val_loss: 2.5381 - val_accuracy: 0.6437
Epoch 228/1000
201/201 [==============================] - 0s 125us/step - loss: 0.0893 - accuracy: 0.9701 - val_loss: 2.6391 - va

...

201/201 [==============================] - 0s 98us/step - loss: 0.0344 - accuracy: 0.9900 - val_loss: 4.1801 - val_accuracy: 0.6207
Epoch 775/1000
201/201 [==============================] - 0s 107us/step - loss: 0.0323 - accuracy: 0.9801 - val_loss: 4.1031 - val_accuracy: 0.6207
Epoch 776/1000
201/201 [==============================] - 0s 108us/step - loss: 0.0342 - accuracy: 0.9900 - val_loss: 4.1430 - val_accuracy: 0.6092
Epoch 777/1000
201/201 [==============================] - 0s 99us/step - loss: 0.0316 - accuracy: 0.9851 - val_loss: 4.0839 - val_accuracy: 0.6207
Epoch 778/1000
201/201 [==============================] - 0s 96us/step - loss: 0.0321 - accuracy: 0.9900 - val_loss: 4.0301 - val_accuracy: 0.5977
Epoch 779/1000
201/201 [==============================] - 0s 97us/step - loss: 0.0420 - accuracy: 0.9851 - val_loss: 4.2484 - val_accuracy: 0.6092
Epoch 780/1000
201/201 [==============================] - 0s 97us/step - loss: 0.0339 - accuracy: 0.9900 - val_loss: 4.2356 - val_a

201/201 [==============================] - 0s 101us/step - loss: 0.0393 - accuracy: 0.9900 - val_loss: 4.3282 - val_accuracy: 0.6092
Epoch 830/1000
201/201 [==============================] - 0s 112us/step - loss: 0.0362 - accuracy: 0.9900 - val_loss: 4.4927 - val_accuracy: 0.6207
Epoch 831/1000
201/201 [==============================] - 0s 102us/step - loss: 0.0439 - accuracy: 0.9801 - val_loss: 4.7508 - val_accuracy: 0.6437
Epoch 832/1000
201/201 [==============================] - 0s 98us/step - loss: 0.0369 - accuracy: 0.9851 - val_loss: 4.4113 - val_accuracy: 0.5977
Epoch 833/1000
201/201 [==============================] - 0s 104us/step - loss: 0.0335 - accuracy: 0.9900 - val_loss: 4.5052 - val_accuracy: 0.6092
Epoch 834/1000
201/201 [==============================] - 0s 107us/step - loss: 0.0274 - accuracy: 0.9900 - val_loss: 4.6064 - val_accuracy: 0.6092
Epoch 835/1000
201/201 [==============================] - 0s 116us/step - loss: 0.0296 - accuracy: 0.9900 - val_loss: 4.5607 - v

201/201 [==============================] - 0s 98us/step - loss: 0.2957 - accuracy: 0.9154 - val_loss: 5.2985 - val_accuracy: 0.6897
Epoch 885/1000
201/201 [==============================] - 0s 104us/step - loss: 0.2334 - accuracy: 0.9353 - val_loss: 3.9059 - val_accuracy: 0.5862
Epoch 886/1000
201/201 [==============================] - 0s 103us/step - loss: 0.5681 - accuracy: 0.9453 - val_loss: 4.3746 - val_accuracy: 0.5977
Epoch 887/1000
201/201 [==============================] - 0s 98us/step - loss: 0.1189 - accuracy: 0.9602 - val_loss: 5.6599 - val_accuracy: 0.6092
Epoch 888/1000
201/201 [==============================] - 0s 95us/step - loss: 0.1536 - accuracy: 0.9502 - val_loss: 5.4387 - val_accuracy: 0.5977
Epoch 889/1000
201/201 [==============================] - 0s 99us/step - loss: 0.2727 - accuracy: 0.9403 - val_loss: 4.5926 - val_accuracy: 0.5287
Epoch 890/1000
201/201 [==============================] - 0s 91us/step - loss: 0.3808 - accuracy: 0.9502 - val_loss: 3.4705 - val_a

201/201 [==============================] - 0s 104us/step - loss: 0.0299 - accuracy: 0.9900 - val_loss: 4.2151 - val_accuracy: 0.5977
Epoch 940/1000
201/201 [==============================] - 0s 112us/step - loss: 0.0450 - accuracy: 0.9801 - val_loss: 4.2283 - val_accuracy: 0.6207
Epoch 941/1000
201/201 [==============================] - 0s 100us/step - loss: 0.0334 - accuracy: 0.9851 - val_loss: 4.3546 - val_accuracy: 0.5977
Epoch 942/1000
201/201 [==============================] - 0s 103us/step - loss: 0.0324 - accuracy: 0.9851 - val_loss: 4.2753 - val_accuracy: 0.6092
Epoch 943/1000
201/201 [==============================] - 0s 96us/step - loss: 0.0261 - accuracy: 0.9900 - val_loss: 4.2837 - val_accuracy: 0.6322
Epoch 944/1000
201/201 [==============================] - 0s 97us/step - loss: 0.0396 - accuracy: 0.9801 - val_loss: 4.5352 - val_accuracy: 0.6207
Epoch 945/1000
201/201 [==============================] - 0s 130us/step - loss: 0.0357 - accuracy: 0.9801 - val_loss: 4.4163 - va

201/201 [==============================] - 0s 101us/step - loss: 0.0323 - accuracy: 0.9851 - val_loss: 4.6406 - val_accuracy: 0.6207
Epoch 995/1000
201/201 [==============================] - 0s 111us/step - loss: 0.0331 - accuracy: 0.9801 - val_loss: 4.4655 - val_accuracy: 0.6207
Epoch 996/1000
201/201 [==============================] - 0s 118us/step - loss: 0.0258 - accuracy: 0.9900 - val_loss: 4.3449 - val_accuracy: 0.6207
Epoch 997/1000
201/201 [==============================] - 0s 103us/step - loss: 0.0304 - accuracy: 0.9900 - val_loss: 4.3161 - val_accuracy: 0.6322
Epoch 998/1000
201/201 [==============================] - 0s 107us/step - loss: 0.0290 - accuracy: 0.9900 - val_loss: 4.4280 - val_accuracy: 0.6207
Epoch 999/1000
201/201 [==============================] - 0s 92us/step - loss: 0.0312 - accuracy: 0.9851 - val_loss: 4.4452 - val_accuracy: 0.6207
Epoch 1000/1000
201/201 [==============================] - 0s 100us/step - loss: 0.0251 - accuracy: 0.9950 - val_loss: 4.3061 - 

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

87/87 [==============================] - 0s 107us/step
over-sampling test accuracy: 63.22%


In [37]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [38]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 68 samples, validate on 30 samples
Epoch 1/100
68/68 [==============================] - 0s 1ms/step - loss: 5.3205 - accuracy: 0.4559 - val_loss: 9.7459 - val_accuracy: 0.6667
Epoch 2/100
68/68 [==============================] - 0s 127us/step - loss: 4.6759 - accuracy: 0.6471 - val_loss: 8.9197 - val_accuracy: 0.8000
Epoch 3/100
68/68 [==============================] - 0s 223us/step - loss: 4.2828 - accuracy: 0.8235 - val_loss: 8.6307 - val_accuracy: 0.7000
Epoch 4/100
68/68 [==============================] - 0s 250us/step - loss: 4.0202 - accuracy: 0.8824 - val_loss: 8.0268 - val_accuracy: 0.8333
Epoch 5/100
68/68 [==============================] - 0s 241us/step - loss: 3.6808 - accuracy: 0.9265 - val_loss: 7.4099 - val_accuracy: 0.9000
Epoch 6/100
68/68 [==============================] - 0s 232us/step - loss: 3.4440 - accuracy: 0.8971 - val_loss: 6.9153 - val_accuracy: 0.9000
Epoch 7/100
68/68 [==============================] - 0s 498us/step - loss: 3.2179 - accuracy: 0.8971

Epoch 58/100
68/68 [==============================] - 0s 259us/step - loss: 0.0806 - accuracy: 0.9853 - val_loss: 0.1697 - val_accuracy: 0.9333
Epoch 59/100
68/68 [==============================] - 0s 261us/step - loss: 0.0806 - accuracy: 0.9559 - val_loss: 0.1645 - val_accuracy: 0.9667
Epoch 60/100
68/68 [==============================] - 0s 208us/step - loss: 0.0805 - accuracy: 0.9559 - val_loss: 0.1618 - val_accuracy: 0.9667
Epoch 61/100
68/68 [==============================] - 0s 213us/step - loss: 0.0791 - accuracy: 0.9559 - val_loss: 0.1670 - val_accuracy: 0.9333
Epoch 62/100
68/68 [==============================] - 0s 204us/step - loss: 0.0761 - accuracy: 0.9853 - val_loss: 0.2040 - val_accuracy: 0.9333
Epoch 63/100
68/68 [==============================] - 0s 198us/step - loss: 0.0754 - accuracy: 1.0000 - val_loss: 0.2366 - val_accuracy: 0.9333
Epoch 64/100
68/68 [==============================] - 0s 235us/step - loss: 0.0755 - accuracy: 1.0000 - val_loss: 0.2543 - val_accuracy:

In [40]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

30/30 [==============================] - 0s 245us/step
combination test accuracy: 93.33%


In [41]:
#### add dropout and regularizor
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [42]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 68 samples, validate on 30 samples
Epoch 1/100
68/68 [==============================] - 0s 2ms/step - loss: 8.5926 - accuracy: 0.2059 - val_loss: 12.2283 - val_accuracy: 0.3000
Epoch 2/100
68/68 [==============================] - 0s 171us/step - loss: 7.7490 - accuracy: 0.3971 - val_loss: 11.0763 - val_accuracy: 0.5667
Epoch 3/100
68/68 [==============================] - 0s 186us/step - loss: 7.4035 - accuracy: 0.4706 - val_loss: 9.0079 - val_accuracy: 0.5667
Epoch 4/100
68/68 [==============================] - 0s 248us/step - loss: 5.6950 - accuracy: 0.6029 - val_loss: 7.8627 - val_accuracy: 0.7000
Epoch 5/100
68/68 [==============================] - 0s 237us/step - loss: 5.3386 - accuracy: 0.6324 - val_loss: 7.9119 - val_accuracy: 0.9000
Epoch 6/100
68/68 [==============================] - 0s 251us/step - loss: 6.9204 - accuracy: 0.6765 - val_loss: 7.8267 - val_accuracy: 0.7000
Epoch 7/100
68/68 [==============================] - 0s 267us/step - loss: 5.5886 - accuracy: 0.67

68/68 [==============================] - 0s 248us/step - loss: 2.7240 - accuracy: 0.7941 - val_loss: 4.1245 - val_accuracy: 0.9000
Epoch 58/100
68/68 [==============================] - 0s 556us/step - loss: 3.2081 - accuracy: 0.6912 - val_loss: 4.0817 - val_accuracy: 0.9000
Epoch 59/100
68/68 [==============================] - 0s 300us/step - loss: 3.3868 - accuracy: 0.6912 - val_loss: 4.0280 - val_accuracy: 0.9000
Epoch 60/100
68/68 [==============================] - 0s 268us/step - loss: 3.5749 - accuracy: 0.7059 - val_loss: 3.9550 - val_accuracy: 0.9000
Epoch 61/100
68/68 [==============================] - 0s 221us/step - loss: 2.8606 - accuracy: 0.7647 - val_loss: 3.8833 - val_accuracy: 0.9000
Epoch 62/100
68/68 [==============================] - 0s 217us/step - loss: 2.8001 - accuracy: 0.8088 - val_loss: 3.8083 - val_accuracy: 0.9000
Epoch 63/100
68/68 [==============================] - 0s 211us/step - loss: 3.2703 - accuracy: 0.7059 - val_loss: 3.7325 - val_accuracy: 0.9000
Epoch

In [44]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

30/30 [==============================] - 0s 111us/step
combination test accuracy: 90.00%


In [45]:
############## Random Forest ##############

In [46]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 54.02%


In [48]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 86.67%


In [50]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.5        0.64285714 0.58974359 0.61538462 0.66666667]
0.6029304029304029


In [51]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         0.93333333 0.92307692 1.         1.        ]
0.9712820512820514
